In [ ]:
import json
import re
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter

In [1]:
#download destination and requirement reports from rp-dev mail chain
#read destination report

dest_rep = pd.read_csv('destination_path')     #enter correct path

#convert requirement report extension from .tsv to .csv on your local device

req_rep = pd.read_csv('requirement_path', encoding="utf-8")     #enter correct path

In [ ]:
#print head of both files to check correct reading process

dest_rep.head()

In [ ]:
req_rep.head()

PART 1 : IDENTIFYING ERRORS FROM REQUIREMENT REPORT AND QUANTIFYING THEM


In [ ]:
#keep only errored stn creations in req report

req_rep = req_rep[req_rep['state'] == 'ERRORED']

In [ ]:
#extract error code and error message from Errors json in requirement report

def extract_error_details_fixed_v3(error_str):
    try:
        if not isinstance(error_str, str) or error_str.strip() == "":
            return "", ""
        cleaned_str = re.sub(r"^\['|'\]$", "", error_str.strip())
        cleaned_str = cleaned_str.replace("'", "\"")
        error_list = json.loads(cleaned_str)

        if isinstance(error_list, list) and len(error_list) > 0:
            error_code = error_list[0].get("code", "").strip()
            error_message = error_list[0].get("message", "").strip()
            return error_code, error_message

    except json.JSONDecodeError:
        return "JSON_ERROR", error_str
    except Exception as e:
        return "PARSE_ERROR", str(e)

req_rep["error_code"], req_rep["error_message"] = zip(*req_rep["errors"].astype(str).map(extract_error_details_fixed_v3))


output_path = "req_rep_errored.csv"
req_rep.to_csv(output_path, index=False, encoding="utf-8")

print(req_rep[["errors", "error_code", "error_message"]].head())



In [2]:
#count unique_errors and their frequency

error_counts = req_rep["error_code"].value_counts().reset_index()
error_counts.columns = ["error_name", "error_count"]

req_rep["error_name"] = req_rep["error_code"]
req_rep["error_count"] = req_rep["error_name"].map(dict(zip(error_counts["error_name"], error_counts["error_count"])))

output_path = "req_rep_error_quantified.csv"
req_rep.to_csv(output_path, index=False)

PART 2 : CHECKING QUANTITY CREATION GAPS IN MASTER POLICY AND DESTINATION REPORT


In [ ]:
#read master policy

mp = pd.read.csv('master_policy_path')


In [ ]:
#keeping only relevant columns from master policy

mp = mp[['fsn', 'use_case', 'cluster', 'inventory_target']]

In [ ]:
#keep only those rows in master policy where inventory target > 0

mp = mp[mp['inventory_target'] > 0]

In [ ]:
#convert cluster column entries from lower to upper case for easy merging

mp['cluster'] = mp['cluster'].str.upper()

In [ ]:
#make unique key for merging with destination report

mp['unique_key'] = mp['fsn'].astype(str) + '_' + mp['cluster'].astype(str)

In [ ]:
#choose use_case according to phase

#phase 2 : hl_nl_ds
#phase 3 : nl_ds
#phase 4 hl_nl_ds_02
#phase 5 : nl_ds2
#phase 6 : hl_ds_adhoc

#for example , taken nl_ds2 for phase 5
mp = mp[mp['use_case'].isin(['nl_ds2'])]


In [ ]:
#make unique key in destination report

dest_rep['unique_key'] = dest_rep['fsn'].astype(str) + '_' + dest_rep['cluster'].astype(str)

#convert unique key to upper case

dest_rep['unique_key'] = dest_rep['unique_key'].str.upper()

#rename a column to clearly analyze after successfull merge

dest_rep.rename(columns={"inventorytarget": "inventory_target_dr"}, inplace=True)

In [ ]:
#in dest_rep count the number of rows which have quantity NaN and also extract those rows and create a new file called dest _rep_qty_nan csv

nan_quantity_count = dest_rep['quantity'].isna().sum()
print(f"Number of rows with NaN in quantity: {nan_quantity_count}")

# Extract rows with NaN in 'quantity' and save to a new file

dest_rep_qty_nan = dest_rep[dest_rep['quantity'].isna()]
dest_rep_qty_nan.to_csv('dest_rep_qty_nan.csv', index=False)

In [ ]:
#merging destination report and master policy to identify gaps

merged_df = pd.merge(mp, dest_rep, on='unique_key', how='left')

In [ ]:
#count the inventory target in master policy and the inventory target in the destination report to see the gap in creation

inventory_target_sum = merged_df['inventory_target'].sum()
inventory_target_dr_sum = merged_df['inventory_target_dr'].sum() if 'inventory_target_dr' in merged_df.columns else 0

print(f"Sum of inventory_target_mp: {inventory_target_sum}")
print(f"Sum of inventory_target_dr: {inventory_target_dr_sum}")

In [ ]:
#putting a check to only see those stn'd where quantity was proposed in the master policy but nothing generated in destination report

merged_df = merged_df[merged_df['inventory_target'] != merged_df['inventory_target_dr']]

In [ ]:
#create a new csv to store the rows where qty is present in master policy but missing in destination report.

merged_df.to_csv('in_mp_not_in_dr.csv')

PART 3 : USING NETWORK LANE POLICY AND MASTER POLICY TO CHECK IF ANY PHASE MISMATCHES ARE THERE IN PHASE 3 & 5

RUN THIS FOR ONLY PHASE 3 & PHASE 5 REPORTS


In [ ]:
#read network lane policy after downloading from gcp

net_lane_pol = pd.read_csv('network_lane_policy_path') #enter correct path

In [ ]:
#choose required columns only

net_lane_pol = net_lane_pol[['Destination Cluster', 'Phase']]

In [ ]:
#merging master policy with network lane policy to map phases from both files and finding mismatch

merged_df_1 = pd.merge(mp, net_lane_pol, left_on='cluster', right_on='Destination Cluster', how='left')

In [ ]:
#keeping only phase 3 and 5 rows because only they might have a mismatch problem

merged_df_1 = merged_df[merged_df_1['use_case'].isin(['nl_ds', 'nl_ds2'])]

In [ ]:
#create a new column called phase_master_policy , and pupulate the column using the use_case

merged_df_1['phase_mp'] = np.where(merged_df_1['use_case'] == 'nl_ds', 3, 5)

In [ ]:
#keep only those rows where there is mismatch in new dataframe called merged_df_mismatch

merged_df_1_mismatch = merged_df_1[ ((merged_df_1['Phase'] == 3) & (merged_df_1['phase_mp'] != 3)) | ((merged_df_1['Phase'] == 5) & (merged_df_1['phase_mp'] != 5)) ]

In [ ]:
#keep unique incorrect phase matches

merged_df_1_mismatch = merged_df_1_mismatch.drop_duplicates(subset=['cluster'])

In [ ]:
#save to csv

merged_df_1_mismatch.to_csv('mismatch_phase_nl_mp.csv')

ONCE ALL OF THIS IS DONE :

UPLOAD :
1.   mismatch_phase_nl_mp.csv
2.   in_mp_not_in_dr.csv
3.   req_rep_error_quantified.csv


